In [39]:
import numpy as np
import matplotlib.pyplot as plt
import itertools
from PMTK.random.preferences_sampler import sample_preferences_from_order, sample_preferences_from_complete_order
from PMTK.utils import *
from PMTK.random.subset_samplers import sample_subsets
from PMTK.utility.additive_utility import AdditiveUtility
from PMTK.preferences import *
from PMTK.utility.utility_fitter import Utility_Fitter
#from PMTK.utility.connivence_solver import Connivence_Solver
from PMTK.utility.extension_solver import *
from tqdm.notebook import tqdm
from itertools import chain

In [40]:
class Connivence_Solver:

    def __init__(self, preferences = None, model = None, solver = None):
        self.preferences = preferences
        self.model = model
        self.problem = None
        if solver == None:
            solver = cp.GLPK_MI
        self.solver = solver
        
        
    def set_preferences(self, preferences):
        self.preferences = preferences
        return self

    def check_connivences(self, min_size = 1, log_output = False):
        #print("Model size:", len(self.model))
        #print("préférences in connivence:", len(self.preferences))

        variables = cp.Variable(len(self.preferences), integer = True)

        mat = self.preferences.get_matrix(self.model, self.preferences.preferred)
        sizes_pref = [len(x[0]) + len(x[1]) for x in self.preferences.preferred]

        mat2 = self.preferences.get_matrix(self.model, self.preferences.indifferent)
        sizes_pref += [len(x[0]) + len(x[1]) for x in self.preferences.indifferent]

        if len(mat2.shape) > 1:
            mat = np.vstack([mat, mat2]).astype(float)
        #print("Constraints matrix", mat.shape)
        mat = mat.T
        #print("Mat shape:", mat.shape)
        #print("Objective:", sum(variables))
        
        
        obj = cp.Minimize( sum( [ variables[i]*(sizes_pref[i]) for i in range(len(self.preferences))] ) )
        #obj = cp.Minimize( sum( [ variables[i]*(random.random()) for i in range(len(self.preferences))] ) )
        #obj = cp.Minimize(0)

        
        problem = cp.Problem(obj, [variables >= 0, sum(variables) >= min_size, mat.astype(float) @ variables == 0.0] )
        p = problem.solve(self.solver, reoptimize=True, verbose=False)
        #print(problem.status)
        #print(problem.status)
        #print(problem.value)
        
        if problem.status == cp.OPTIMAL:
            connivent = []
            for i in np.where(variables.value != 0)[0]:
                for k in range(int(variables.value[i])):
                    connivent.append(self.preferences[i])
            return connivent
        
        assert problem.status == cp.INFEASIBLE
        
        if problem.status == cp.INFEASIBLE:
            return []

In [41]:
class Kernel_Finder:
    
    def __init__(self,items, preferences, model, epsilon = 1e-6, solver = None):
        self.items = items
        self.preferences = preferences
        self.model = model
        self.epsilon = epsilon
        #Solver variables
        self.vars = None
        self.is_not_zero = None
        self.cst = None
        self.additivity = None
        if not solver:
            solver = cp.GLPK_MI
        self.solver = solver
    
    def build_program(self):
        self.vars = cp.Variable(len(self.model))
        self.is_not_zero = cp.Variable(len(self.model), integer = True)
        self.additivity = cp.Variable()
        
        self.cst= [self.is_not_zero >= 0, self.is_not_zero <= 1]
        self.cst.append(self.additivity >= 0)
        
        
        for p in self.preferences.preferred:
            p_v = vectorize_subset(p[0], self.model) - vectorize_subset(p[1], self.model) 
            cst = self.vars @ p_v >= self.epsilon
            self.cst.append(cst)
            
        for p in self.preferences.indifferent:
            p_v = vectorize_subset(p[0], self.model) - vectorize_subset(p[1], self.model) 
            cst = self.vars @ p_v == 0
            self.cst.append(cst)
            
        #self.cst.append(self.vars <= 1)
        #self.cst.append(self.vars >= -1)
        self.cst.append(self.vars <= self.is_not_zero)
        self.cst.append(self.vars >= -self.is_not_zero)
        length_vector = np.array([len(s) for s in self.model])

        for nz, size in zip(self.is_not_zero, length_vector):
            self.cst.append(self.additivity >= nz*size)
            pass
        
    def compute_kernel(self):
        self.cardinality_obj = cp.Minimize(sum(self.is_not_zero))
        self.additivity_obj =  cp.Minimize(self.additivity)
        try:
            prob = cp.Problem(self.additivity_obj, self.cst)
            prob.solve(self.solver, reoptimize=True)
            #print(f"First solution: ({prob.status}, {self.additivity.value},  {sum(self.is_not_zero).value})")
            self.cst.append(self.additivity == self.additivity.value)
            prob = cp.Problem(self.cardinality_obj, self.cst)
            prob.solve(self.solver, reoptimize=True)
            #print(f"Second solution:  ({prob.status}, {self.additivity.value},  {sum(self.is_not_zero).value})") 
            self.kernel = []
            for i,v in enumerate(self.vars):
                if v.value != 0:
                    self.kernel.append(self.model[i])
            return self.kernel
            #return self.model
        except (Exception):
            print('[!!]', end = "")
            return self.model

n_items = 12
n_preferences = 100
items = np.arange(n_items)
pref = sample_preferences_from_order(items, n_preferences)
theta = get_all_k_sets(items, 2)
KF = Kernel_Finder(items, pref, theta)
KF.build_program()
theta = KF.compute_kernel()
print("Theta:", theta)
KF = Kernel_Finder(items, pref, theta)
KF.build_program()
theta = KF.compute_kernel()
print("Theta:", theta)

In [42]:
class Candidate_Iterator:
    
    def __init__(self, items, subsets, banned = None, max_size = None, min_size = 1, pred = None):
        if not max_size:
            max_size = len(items)
        if not banned:
            banned=[]
        if not pred:
            pred = lambda x:True
        self.items = items
        self.subsets = subsets
        self.banned = banned
        self.min_size = min_size
        self.max_size = max_size
        self.pred = pred
        self.comb_it = []
        self.generated = []
        for k in range(1, len(self.items)):
            its = []
            for s in subsets:
                if len(s) >= k:
                    its.append(itertools.combinations(s, k))
            self.comb_it.append(chain(*its))
        self.k = 1
        self.id = 0
    
    def __iter__(self):
        return self
    
    def ban(self, *els):
        for el in els:
            self.banned += [tuple(sorted(el))]
    
    def __next__(self):
        self.k = max(self.k, self.min_size)
        while self.k <= min(self.max_size, len(self.items)-1):
            for c in self.comb_it[self.k-1]:
                if not c in self.banned and self.pred(c) and not c in self.generated:
                    self.generated.append(c)
                    return c
            self.k += 1
        raise StopIteration

Et la avec les cuts: 

In [43]:
class Node:
    
    def __init__(self, tree, theta, banned, level = 0):
        self.theta = theta
        self.k = additivity(theta)
        self.s = len(theta)
        self.tree = tree
        self.banned = banned
        self.children = []
        self.connivent = None
        self.level = level
        self.found_connivent = False
        self.solved = False
    
    def get_next_child(self, min_size = None, max_size = None):
        if not self.connivent:
            self.compute_connivent(self.tree.preferences)
        try:
            if min_size:
                self.child_iterator.min_size = min_size
            if max_size:
                self.child_iterator.max_size = max_size
            e = next(self.__it)
            return e
        except StopIteration:
            return None
        
    def __str__(self):
        c = f"-----Node({self.theta}) [Banned = {self.banned}]\n"
        for i in self.children:
            c += self.level*"\t"+ f"{i}"
        return c
        
    
    def __repr__(self):
        return self.__str__()
    
    
    def open_node(self):
        if dominated(self.theta, self.tree.found_theta):
            print("#", end="")
            return
        
        print(f".", end="")
        
        if not self.found_connivent:
            self.connivent = self.tree.found_connivence(self.theta)
            self.found_connivent = True
            
        if not self.connivent:
            self.solved = True
            return 
            
        subsets = []
        for x,y in self.connivent:
            if not x in subsets:
                subsets.append(x)
            if not y in subsets:
                subsets.append(y)
        self.child_iterator = Candidate_Iterator(self.tree.items,subsets,banned = self.theta + self.banned)
        self.__it = iter(self.child_iterator)
        e = self.get_next_child(min_size = 0, max_size = self.tree.get_additivity())
        all_bans = []
        cpt  = 0
        while e:
            cpt = cpt + 1
            if e in all_bans:
                continue
            #print(f"Node:{self.theta} with id: ", self.idx," -> son's id :", str(self.idx + f".{cpt}"))
            child = Node(self.tree, self.theta + [e], banned =list(set( self.banned + list(all_bans))) , level = self.level + 1)
            #print(f"..{cpt}..", end="")
            child.open_node()
            
            all_bans.append(e)
            e = self.get_next_child(min_size = 0, max_size = self.tree.get_additivity())
            
            self.children.append(child)
            
    
class Tree:
    def __init__(self, items, preferences, init_theta, epsilon=1e-4):
        self.items = items
        self.preferences = preferences
        self.found_theta = []
        self.connivent_calculated = []  
        self.head = Node(self, init_theta, [])
        self.epsilon = epsilon
        
    def __str__(self):
        c = f"==========Theta Tree===== \n"
        c += str(self.head)
        return c
    
    def get_additivity(self):
        if len(self.found_theta)>0:
            return additivity(self.found_theta[0])
        return np.inf

    def found_connivence(self, theta):
        for connivent in self.connivent_calculated:
            if self.is_connivent(connivent, theta):
                return connivent
        CS = Connivence_Solver(self.preferences, theta)        
        #print("Solving...", end="")
        c = CS.check_connivences()
        #print(f"  Solved in {(time.time() - t):.2f} s    ",end="")
        if not c:
            self.found_theta.append(theta)
            #print(f"Found theta: {additivity(theta)} , {len(theta)}") 
            KF = Kernel_Finder(self.items, self.preferences, theta, epsilon=self.epsilon)
            KF.build_program()
            kernel = KF.compute_kernel()
            self.found_theta.append(kernel)
            self.found_theta = keep_non_dominated(self.found_theta)
            #print("Found:",theta)
            #print("New size: ", len(self.found_theta))
            print(f"! {len(self.found_theta)} \n \n ")
            return c
        #if not c in self.connivent_calculated:
        #    self.connivent_calculated.append(c)
        return c
    
    def is_connivent(self, preference_set, theta):
        L = []
        for x,y in preference_set:
            L.append(vectorize_preference(x,y,theta))
        L = np.array(L)
        return ((L.sum(axis=0) == 0).all())
        

In [46]:
n_items = 7
n_preferences = 100
it = np.arange(n_items)
p = sample_preferences_from_order(it, n_preferences, indifference_rate=0).sort_by_n_candidates()
T = Tree(it, p,get_all_k_sets(it, 1))

In [47]:
T.head.open_node()

.Long-step dual simplex will be used
..Long-step dual simplex will be used
.Long-step dual simplex will be used
.Long-step dual simplex will be used
Long-step dual simplex will be used
.Long-step dual simplex will be used
..Long-step dual simplex will be used
.Long-step dual simplex will be used
.Long-step dual simplex will be used
.Long-step dual simplex will be used
.Long-step dual simplex will be used
.Long-step dual simplex will be used
Long-step dual simplex will be used
Long-step dual simplex will be used
! 1 
 
 
###########################################################################################################.#Long-step dual simplex will be used
##########.#Long-step dual simplex will be used
#########.#Long-step dual simplex will be used
########.#Long-step dual simplex will be used
#######.##Long-step dual simplex will be used
#####.#Long-step dual simplex will be used
#####.#Long-step dual simplex will be used
####.Long-step dual simplex will be used
#########.Long-

In [48]:
for i in T.found_theta:
    print(i)

[(1,), (3,), (5,), (6,), (0, 1), (4, 5), (1, 3), (3, 5), (2, 5), (2, 6)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (0, 5), (1, 3), (2, 6)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (0, 5), (1, 3), (2, 6)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (0, 5), (1, 3), (1, 2)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (0, 5), (1, 3), (1, 2)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (0, 5), (1, 3), (2, 3)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (0, 5), (1, 3), (2, 3)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (0, 5), (1, 3), (3, 6)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (0, 5), (1, 3), (3, 6)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (1, 3), (1, 5), (0, 2)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (1, 3), (1, 5), (0, 2)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (1, 3), (3, 5), (2, 6)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (1, 3), (3, 5), (2, 6)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (1, 3), (3, 5), (1, 2)]
[(0,), (1,), (2,), (3,), (4,), (5,), (6,), (1, 3), (3, 5

In [49]:
union(T.found_theta)

[(1,),
 (3,),
 (5,),
 (6,),
 (0, 1),
 (4, 5),
 (1, 3),
 (3, 5),
 (2, 5),
 (2, 6),
 (0,),
 (2,),
 (4,),
 (0, 5),
 (1, 2),
 (2, 3),
 (3, 6),
 (1, 5),
 (0, 2),
 (5, 6)]